In [1]:
import requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import pandas as pd
import requests
import re

In [4]:
def scrape_glovo_restaurants(url_template, pages):
    all_data = []

    for page_num in range(1, pages+1):
        url = url_template.format(page_num)
        html = requests.get(url)
        soup = BeautifulSoup(html.content, "html.parser")
# First we get the name of the restaurant
        tags_name = soup.find_all('h3', attrs = {"class":"card-title"})
        names = [i.getText().strip() for i in tags_name]
# Then the type of cuisine the restaurant serves, this can be more than 1 style
        cuisine_style = soup.find_all('div', attrs = {"class":"store-filter-container"})
        cuisine_style = [i.getText().replace("\n", "").strip().split('                      ') for i in cuisine_style]
# Then the rating (from 0-100%, 100% being the highest rating)
        rating = soup.find_all('span', attrs = {"class":"store-card-rating-info__label"})
        rating = [i.getText().strip() for i in rating]
# Amount of reviews, 500+ being the highest amount
        number_reviews = soup.find_all('span', attrs = {"class":"store-card-rating-info__label--secondary"})
        number_reviews = [re.search(r'\((.*?)\)', i.getText().strip()).group(1) for i in number_reviews]

# Adding all information to a dataframe
        for name, style, rating, num_reviews in zip(names, cuisine_style, rating, number_reviews):
            data = {
                'Name': name,
                'Cuisine Style': style,
                'Rating Glovo': rating,
                'Number of Reviews': num_reviews
            }
            all_data.append(data)

    df = pd.DataFrame(all_data)
    return df


In [5]:
url_template = "https://glovoapp.com/es/en/barcelona/restaurants_1/?page={}"

df_glovo = scrape_glovo_restaurants(url_template, 79)

In [6]:
df_glovo

,Name,Cuisine Style,Rating Glovo,Number of Reviews
0,VICIO,[Burgers],94%,500+
1,KFC,"[American, Chicken]",95%,43
2,Healthy Poke,"[Healthy, Poke]",97%,500+
3,Gonzalez & Co,"[American, Latin American]",97%,500+
4,Café & Desayuno,[Breakfast],95%,500+
...,...,...,...,...
1511,Pasta Sicily,"[Burgers, Italian]",44%,19
1512,Sushi Style,"[Japanese, Sushi]",54%,49
1513,Green Sushi,"[Asian, Japanese]",41%,16
1514,Pulled Burger,"[American, Burgers]",28%,16


In [7]:
df_glovo.to_csv(r'../../data/df_glovo.csv')